<a href="https://colab.research.google.com/github/TBab0/duth_cv4/blob/main/tuned_hw3b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Declare Imports
from keras import models, regularizers, layers, optimizers, losses, metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils, to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
import os
from google.colab import drive
import zipfile
import tensorflow as tf
import datetime

In [2]:
drive.mount('/content/drive')
local_zip = '/content/drive/MyDrive/imagedb_caltech.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

train_dir = '/content/caltech/imagedb'
test_dir = '/content/caltech/imagedb_test'

Mounted at /content/drive


In [3]:
# Prep the Train Valid and Test directories for the generator
batch_size = 128
row, col, cha = 256, 256, 3
target_size = (row, col)

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   horizontal_flip=True,
                                   validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training', seed=1) # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',  seed=1) # set as validation data

Found 388 images belonging to 5 classes.
Found 94 images belonging to 5 classes.


In [4]:
# Convoluted Base MODEL
conv_base = ResNet50(weights='imagenet', include_top=False, input_shape=(row, col, cha))
# print(conv_base.summary())

# Freeze the layers
for layer in conv_base.layers[:]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
# for layer in conv_base.layers:
    # print(layer, layer.trainable)
     

94765736/94765736 [==============================] - 5s 0us/step


In [5]:
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(conv_base)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(2048, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                                 
 dense (Dense)               (None, 2048)              268437504 
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               1049088   
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 5)                 2

In [6]:
callbacks = []

save_best_callback = tf.keras.callbacks.ModelCheckpoint(f'best_weights.hdf5', save_best_only=True, verbose=1)
callbacks.append(save_best_callback)

early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, verbose=1)
callbacks.append(early_stop_callback)

In [7]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=optimizers.adam_v2.Adam(learning_rate=1e-4), metrics=['acc'])
# Train the model
history = model.fit_generator(train_generator, steps_per_epoch=train_generator.samples/train_generator.batch_size, epochs=30, validation_data=validation_generator, verbose=1, callbacks = callbacks)
 
# Save the model
model.save('small_last4.h5')

<ipython-input-7-24e1b0ab929e>:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, steps_per_epoch=train_generator.samples/train_generator.batch_size, epochs=30, validation_data=validation_generator, verbose=1, callbacks = callbacks)


Epoch 1/30
4/3 [=======================================] - ETA: 0s - loss: 3.4111 - acc: 0.5567
Epoch 1: val_loss improved from inf to 0.16488, saving model to best_weights.hdf5
3/3 [==============================] - 34s 11s/step - loss: 3.4111 - acc: 0.5567 - val_loss: 0.1649 - val_acc: 0.9574
Epoch 2/30
4/3 [=======================================] - ETA: 0s - loss: 0.3869 - acc: 0.9639
Epoch 2: val_loss improved from 0.16488 to 0.00009, saving model to best_weights.hdf5
3/3 [==============================] - 18s 9s/step - loss: 0.3869 - acc: 0.9639 - val_loss: 8.6786e-05 - val_acc: 1.0000
Epoch 3/30
4/3 [=======================================] - ETA: 0s - loss: 0.0940 - acc: 0.9897
Epoch 3: val_loss did not improve from 0.00009
3/3 [==============================] - 3s 1s/step - loss: 0.0940 - acc: 0.9897 - val_loss: 0.0246 - val_acc: 0.9787
Epoch 4/30
4/3 [=======================================] - ETA: 0s - loss: 0.1630 - acc: 0.9768
Epoch 4: val_loss did not improve from 0.00009

In [8]:
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
test_datagen  = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator =  test_datagen.flow_from_directory(test_dir,
                                                        batch_size=128,
                                                        class_mode='categorical',
                                                        #  color_mode='grayscale',
                                                         target_size=target_size) 
loss, acc = model.evaluate(test_generator)

Found 52 images belonging to 5 classes.
1/1 [==============================] - 2s 2s/step - loss: 0.0000e+00 - acc: 1.0000
